In [7]:
import pandas as pd
import plotly.express as px

In [8]:
df = pd.read_pickle("results.pkl")
df["nodes"] = df["nodes"].astype(str)
df["top_k"] = df["top_k"].astype(str)
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x))
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x))
# df["Path Selector"] = df["path_selector_class"]

In [9]:
df.to_csv("results.csv")

In [10]:
df.columns

Index(['Trial Number', 'Original Distance', 'Time Taken', 'Perturbations',
       'Total Perturbation', 'Success', 'experiment_type',
       'perturbation_function', 'global_budget', 'local_budget', 'epsilon',
       'k', 'top_k', 'max_iterations', 'graph_name', 'weights', 'nodes',
       'source', 'target', 'goal', 'Path Selector', 'Number of Paths',
       'Number of Edges', 'Add Times', 'Perturb Times', 'Iterations',
       'Final Distance', 'Status', 'Average Add Time', 'Average Perturb Time'],
      dtype='object')

In [13]:
px.bar(df, x="nodes", y="Time Taken", barmode="group", color="Path Selector", facet_col="top_k", facet_row="k", pattern_shape="Status", hover_data=["Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations"])